In [1]:
# (0) 필요 모듈 임포트
from datetime import datetime, timedelta
import pandas as pd
import requests
import json
import yaml
import time

# (1) 개인정보 파일 가져오기
with open('config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
ACCESS_TOKEN_EXPIRED = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
URL_BASE = _cfg['URL_BASE']
HTS_ID = _cfg['HTS_ID']
print(APP_KEY, APP_SECRET, ACCESS_TOKEN, HTS_ID)

# (2) 함수 정의
## 1. 접근 토큰 발급
def get_access_token():
    """ OAuth 인증 > 접근토큰발급 """
    headers = {"content-type": "application/json"}
    body = {
        "grant_type": "client_credentials",
        "appkey": APP_KEY,
        "appsecret": APP_SECRET
    }
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"

    time.sleep(0.05)  # 유량제한 예방 (REST: 1초당 20건 제한)
    res = requests.post(URL, headers=headers, data=json.dumps(body))

    if res.status_code == 200:
        try:
            access_token = res.json().get("access_token")
            access_token_expired = res.json().get("access_token_token_expired")  # 수정된 키
            return access_token, access_token_expired
        except KeyError as e:
            print(f"토큰 발급 중 키 에러 발생: {e}")
            print(res.json())
            return None, None
    else:
        print("접근 토큰 발급이 불가능합니다. 응답 코드:", res.status_code)
        print("응답 내용:", res.json())
        return None, None

FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'

In [ ]:
ACCESS_TOKEN, ACCESS_TOKEN_EXPIRED = get_access_token()
ACCESS_TOKEN, ACCESS_TOKEN_EXPIRED

In [33]:
def get_domestic_index_data(index_code, date, access_token):
    """
    국내 지수 데이터를 조회하는 함수
    :param index_code: 조회할 지수 코드 (예: '0001')
    :param date: 조회 기준 날짜 (예: '20241120')
    :param access_token: 인증 토큰
    :return: API 응답 데이터 (JSON)
    """
    url = f"{URL_BASE}/uapi/domestic-stock/v1/quotations/inquire-index-daily-price"
    params = {
        "FID_COND_MRKT_DIV_CODE": "U",  # 시장 구분 코드 (업종: 'U')
        "FID_INPUT_ISCD": index_code,  # 지수 코드
        "FID_INPUT_DATE_1": date,  # 조회 날짜
        "FID_PERIOD_DIV_CODE": "D"  # 기간 분류 코드 ( D:일별 , W:주별, M:월별 )
    }
    headers = {
        'content-type': 'application/json',
        'authorization': f"Bearer {access_token}",
        'appkey': APP_KEY,
        'appsecret': APP_SECRET,
        'tr_id': 'FHPUP02120000',  # 국내업종 일자별지수 TR_ID
        'custtype': 'P'
    }
    print(params)
    # print(f"API 호출: {url}, 파라미터: {params}")
    time.sleep(0.05)  # 유량 제한 방지를 위한 대기

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"API 호출 실패: {response.status_code}, 내용: {response.text}")
        return None

In [35]:
# 3. 호출 반복하여 데이터를 데이터프레임에 저장하는 함수
def fetch_multiple_dates(index_code, start_date, num_calls, access_token):
    # 데이터를 저장할 리스트
    all_data = []
    
    # 첫 호출 날짜 설정
    current_date = start_date
    
    for _ in range(num_calls):
        # 데이터 호출
        result = get_domestic_index_data(index_code, current_date, access_token)
        
        # API 호출 성공 시
        if result and 'output2' in result:
            # output2가 비어있는지 체크
            if not result['output2']:
                print(f"데이터가 없습니다. 날짜: {current_date}")
                break  # 데이터가 없으면 종료
            
            all_data.extend(result['output2'])
            
            # 마지막 데이터의 "stck_bsop_date" 값을 추출하여 이전 날짜로 설정
            last_stck_bsop_date = result['output2'][-1]['stck_bsop_date']
            current_date = (datetime.strptime(last_stck_bsop_date, "%Y%m%d") - timedelta(days=1)).strftime("%Y%m%d")
        else:
            print(f"API 호출 실패 또는 예상 외의 응답: {result}")
            break  # 호출 실패 시 종료
    
    # 결과를 데이터프레임으로 변환
    df = pd.DataFrame(all_data)
    return df

# 엑셀 파일로 내보내는 함수
def save_to_excel(df, index_code, start_date, end_date):
    # 파일 이름 설정 (index_code, 시작 날짜, 끝 날짜)
    file_name = f"{index_code}_daily_data_{start_date}_{end_date}.xlsx"
    
    # 데이터프레임을 엑셀 파일로 저장
    df.to_excel(file_name, index=False)
    print(f"파일 저장 완료: {file_name}")

In [37]:
# 실행부분
index_code = '0001'  # 예시 인덱스 코드
start_date = datetime.today().strftime('%Y%m%d')  # 시작 날짜 = 오늘날짜
num_calls = 200  # 호출할 횟수

# 데이터 수집
df = fetch_multiple_dates(index_code, start_date, num_calls, ACCESS_TOKEN)

# 첫 번째 및 마지막 데이터 일시 가져오기
if not df.empty:
    first_date = df.iloc[0]['stck_bsop_date']
    last_date = df.iloc[-1]['stck_bsop_date']
    save_to_excel(df, index_code, first_date, last_date)
else:
    print("데이터가 없습니다.")

{'FID_COND_MRKT_DIV_CODE': 'U', 'FID_INPUT_ISCD': '0001', 'FID_INPUT_DATE_1': '20241120', 'FID_PERIOD_DIV_CODE': 'D'}
{'FID_COND_MRKT_DIV_CODE': 'U', 'FID_INPUT_ISCD': '0001', 'FID_INPUT_DATE_1': '20240624', 'FID_PERIOD_DIV_CODE': 'D'}
{'FID_COND_MRKT_DIV_CODE': 'U', 'FID_INPUT_ISCD': '0001', 'FID_INPUT_DATE_1': '20240124', 'FID_PERIOD_DIV_CODE': 'D'}
{'FID_COND_MRKT_DIV_CODE': 'U', 'FID_INPUT_ISCD': '0001', 'FID_INPUT_DATE_1': '20230827', 'FID_PERIOD_DIV_CODE': 'D'}
{'FID_COND_MRKT_DIV_CODE': 'U', 'FID_INPUT_ISCD': '0001', 'FID_INPUT_DATE_1': '20230402', 'FID_PERIOD_DIV_CODE': 'D'}
{'FID_COND_MRKT_DIV_CODE': 'U', 'FID_INPUT_ISCD': '0001', 'FID_INPUT_DATE_1': '20221107', 'FID_PERIOD_DIV_CODE': 'D'}
{'FID_COND_MRKT_DIV_CODE': 'U', 'FID_INPUT_ISCD': '0001', 'FID_INPUT_DATE_1': '20220613', 'FID_PERIOD_DIV_CODE': 'D'}
{'FID_COND_MRKT_DIV_CODE': 'U', 'FID_INPUT_ISCD': '0001', 'FID_INPUT_DATE_1': '20220112', 'FID_PERIOD_DIV_CODE': 'D'}
{'FID_COND_MRKT_DIV_CODE': 'U', 'FID_INPUT_ISCD': '0001'

In [46]:
df  # 19830104 ~ 20241120 기간의 코스피지수 일별시세

,stck_bsop_date,bstp_nmix_prpr,prdy_vrss_sign,bstp_nmix_prdy_vrss,bstp_nmix_prdy_ctrt,bstp_nmix_oprc,bstp_nmix_hgpr,bstp_nmix_lwpr,acml_vol_rlim,acml_vol,acml_tr_pbmn,invt_new_psdg,d20_dsrt
0,20241120,2482.29,2,10.34,0.42,2475.76,2489.15,2471.79,100.00,408790,7820085,-18.64,98.05
1,20241119,2471.95,2,2.88,0.12,2469.13,2479.39,2465.15,99.50,410856,9085861,-33.75,97.42
2,20241118,2469.07,2,52.21,2.16,2440.31,2480.01,2437.53,92.09,443926,10621301,-46.05,97.11
3,20241115,2416.86,5,-2.00,-0.08,2413.05,2433.18,2390.56,65.58,623327,12365759,-47.95,94.81
4,20241114,2418.86,2,1.78,0.07,2430.26,2441.43,2410.93,63.87,639990,13017388,-49.05,94.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11071,19830108,120.71,5,-0.38,-0.31,120.71,120.71,120.71,451.70,90500,0,0.00,0.00
11072,19830107,121.09,5,-1.72,-1.40,121.09,121.09,121.09,217.08,188310,0,0.00,0.00
11073,19830106,122.81,2,0.25,0.20,122.81,122.81,122.81,165.37,247200,0,0.00,0.00
11074,19830105,122.56,2,0.04,0.03,122.56,122.56,122.56,259.37,157610,0,0.00,0.00
